In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
from numpy import mean
from numpy import std
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from BaselineRemoval import BaselineRemoval
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report

plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [2]:
ages_df = pd.read_csv("AGEs.csv")
arm_df = pd.read_csv("innerArm.csv")
thumbnail_df = pd.read_csv("thumbNail.csv")
earlobe_df = pd.read_csv("earLobe.csv")
vein_df = pd.read_csv("vein.csv")

In [3]:
def polyFit(df):
    polynomial = 5
    gradient = 0.05
    df
    return df

In [4]:
def crop_range_to_numpy(df):
    
    # Dropping columns : ['patientID', 'has_DM2']
    df_dropped_index = df.drop(labels=['patientID', 'has_DM2'], axis=1, inplace=False)
    
    # Cropping raman shift from range 800 cm-1 to 1800cm-1
    cropped_signal = df_dropped_index.loc[:, 'Var802':'Var1801']
    
    # Convert data to numpy.ndarray
    cropped_signal_np = cropped_signal.to_numpy()
    
    # Dropping the first index corresponding to label : ramanShift
    cropped_signal_np = cropped_signal_np[1:]
    
    return cropped_signal_np

In [5]:
arm_df

,patientID,has_DM2,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var3152,Var3153,Var3154,Var3155,Var3156,Var3157,Var3158,Var3159,Var3160,Var3161
0,ramanShift,NaN,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,...,3150,3151,3152,3153,3154,3155,3156,3157,3158,3159
1,DM201,1.0,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,...,0,0,0,0,0,0,0,0,0,0
2,DM202,1.0,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,214.800000,...,0,0,0,0,0,0,0,0,0,0
3,DM203,1.0,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,457.800000,...,0,0,0,0,0,0,0,0,0,0
4,DM204,1.0,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,181.833333,...,0,0,0,0,0,0,0,0,0,0
5,DM205,1.0,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,179.800000,...,0,0,0,0,0,0,0,0,0,0
6,DM206,1.0,237.400000,237.400000,237.400000,237.400000,237.400000,237.400000,237.400000,237.400000,...,0,0,0,0,0,0,0,0,0,0
7,DM207,1.0,217.333333,217.333333,217.333333,217.333333,217.333333,217.333333,217.333333,217.333333,...,0,0,0,0,0,0,0,0,0,0
8,DM208,1.0,89.166667,89.166667,89.166667,89.166667,89.166667,89.166667,89.166667,89.166667,...,0,0,0,0,0,0,0,0,0,0
9,DM209,1.0,336.600000,336.600000,336.600000,336.600000,336.600000,336.600000,336.600000,336.600000,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = crop_range_to_numpy(arm_df)
y = arm_df['has_DM2'].drop(0, inplace=False).to_numpy()

In [8]:
X

array([[345.8314642 , 350.6934332 , 351.9772339 , ..., 107.9693807 ,
        113.2883358 , 111.4543031 ],
       [232.1680176 , 235.3096008 , 240.2931427 , ...,  76.33356629,
         72.39400024,  72.14946137],
       [482.391803  , 487.6876221 , 490.1527954 , ..., 160.3856354 ,
        164.080011  , 159.1892487 ],
       ...,
       [133.321551  , 137.2859294 , 139.4813639 , ...,  40.94071897,
         46.81751378,  39.37948608],
       [301.1747081 , 306.9342702 , 309.3126323 , ..., 111.7359607 ,
        107.1366704 , 105.9139811 ],
       [170.328982  , 174.9665553 , 179.3695526 , ...,  57.2306633 ,
         56.78333728,  54.74552091]])

In [9]:
# Tensorflow
import tensorflow as tf

from tensorflow import keras
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential

# Metrics evaluator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4, shuffle=True, test_size=0.2)

In [11]:
X_train.shape

(16, 1000)

In [12]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Dense(14, input_dim=1000, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='softmax')
  ])

In [14]:
model = create_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
tf.keras.utils.plot_model(model, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
